# Logo classification on Flickr27 Logo Dataset

In [1]:
import csv
import pandas as pd
import numpy as np
from PIL import Image, ImageOps
import glob
import cv2
from sklearn.preprocessing import LabelEncoder

/home/mukesh/miniconda3/envs/statoil/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Dataset

In [2]:
txt_file = "flickr_logos_27_dataset/flickr_logos_27_dataset_training_set_annotation.txt"
test_file = "flickr_logos_27_dataset/flickr_logos_27_dataset_query_set_annotation.txt"
image_dir = "flickr_logos_27_dataset/flickr_logos_27_dataset_images/"
extra_img = "more_images/"
new_size=(128,128)

In [3]:
annot_train = np.loadtxt(txt_file, dtype='a')
annot_test = np.loadtxt(test_file, dtype='a')

# Pre processing Images

## Major Challenges
Training data has 27 classes but test data set has 28 classes. Test data has an extra class "none". So, I added some new images to training data which do not have logos from SUN 397 dataset.

In [4]:
def parse_train(annot,new_size=new_size):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    subset_class = annot[2].decode('utf-8')
    x1 = int(annot[3].decode('utf-8'))
    y1 = int(annot[4].decode('utf-8'))
    x2 = int(annot[5].decode('utf-8'))
    y2 = int(annot[6].decode('utf-8'))
    img = Image.open(image_dir+fn)
    area = (x1, y1, x2, y2)
    img= img.crop(area)
    img = img.resize((new_size))
    return img, class_name

def parse_test(annot,new_size=new_size):
    fn = annot[0].decode('utf-8')
    class_name = annot[1].decode('utf-8')
    img = Image.open(image_dir+fn)
    img = img.resize((new_size))
    return img, class_name

def new_img_to_train(images,new_size=new_size):
    img = Image.open(images)
    img = img.resize(new_size)
    class_name ='none'
    return img, class_name

In [5]:
class_=[]
images=[]
for annot in annot_train:
    img,class_name = parse_train(annot)
    images.append(img)
    class_.append(class_name)
    
for fn in glob.glob(extra_img+'/*.jpg'):
    img, class_name=new_img_to_train(fn)
    images.append(img)
    class_.append(class_name)
data=pd.DataFrame(data={'image':images,'labels':class_})
print(data.shape)
data.head()

(4616, 2)


,image,labels
0,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
1,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
2,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
3,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
4,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas


In [6]:
class_=[]
images=[]
for annot in annot_test:
    img,class_name = parse_test(annot)
    images.append(img)
    class_.append(class_name)
test_data=pd.DataFrame(data={'image':images,'labels':class_})
test_data.head()

,image,labels
0,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
1,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
2,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
3,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas
4,<PIL.Image.Image image mode=RGB size=128x128 a...,Adidas


In [7]:
def to_OHE(label,nb_classes):
    L_enc = LabelEncoder()
    L_enc.fit(label)
    return np_utils.to_categorical(L_enc.transform(label), nb_classes)

def inverse_OHE(label,encoded_labels,nb_classes):
    decoded=encoded_labels.argmax(1)
    L_enc = LabelEncoder()
    L_enc.fit(label)
    return L_enc.inverse_transform(decoded)

In [8]:
data_labels=to_OHE(data['labels'],len(set(data['labels'])))
test_labels=to_OHE(test_data['labels'],len(set(test_data['labels'])))

In [9]:
tr_images = np.array([np.array(im) for im in data['image']])
test_images = np.array([np.array(im) for im in test_data['image']])

# Create Model and Train

In [10]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Dropout, Activation, Conv2D, MaxPooling2D, Lambda, Flatten, GlobalAveragePooling2D, BatchNormalization
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import RMSprop, nadam, adam
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

In [11]:
def pre_trained_vgg():
    base_model = MobileNet(weights='imagenet', include_top=False, 
                 input_shape=tr_images.shape[1:], classes=len(set(data['labels'])))
    x = base_model.get_layer('conv_pw_13_relu').output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    FC = Dense(128, activation='relu')(x)
    FC = Dropout(0.4)(FC)
    preds = Dense(len(set(data['labels'])), activation='sigmoid')(FC)
    model = Model(inputs=base_model.input,outputs=preds)
    model.compile(loss='binary_crossentropy',metrics=['accuracy'],optimizer=adam(lr=0.001))
    return model

def limit_mem():
    import keras.backend as K
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    print('gpu memory cleaned')

In [12]:
from sklearn.model_selection import KFold
folds=KFold(n_splits=10, random_state=42, shuffle=True)
predict=np.zeros((len(test_images), 28))
for train,test in folds.split(tr_images,data_labels):
    limit_mem()
    X_train, X_test = tr_images[train], tr_images[test]
    Y_train, Y_test = data_labels[train], data_labels[test]
    model=None
    model=pre_trained_vgg()
    hist=model.fit(X_train/255,Y_train, validation_data=(X_test/255, Y_test),epochs=10, batch_size=16,verbose=1)
    preds=model.evaluate(X_test/255,Y_test,batch_size=16, verbose=1)
    print("validation results" ,preds[0], preds[1])
    test_pred = model.evaluate(test_images/255,test_labels,batch_size=16, verbose=1)
    print("test results", test_pred[0], test_pred[1])
    predict +=model.predict(test_images,batch_size=16, verbose=1)
    del test_pred,preds,hist

gpu memory cleaned
Train on 4154 samples, validate on 462 samples
Epoch 1/10
4154/4154 [==============================] - 32s 8ms/step - loss: 0.0985 - acc: 0.9597 - val_loss: 0.0061 - val_acc: 0.9981
Epoch 2/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0130 - acc: 0.9961 - val_loss: 0.0045 - val_acc: 0.9987
Epoch 3/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0065 - acc: 0.9981 - val_loss: 0.0193 - val_acc: 0.9960
Epoch 4/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0053 - acc: 0.9982 - val_loss: 0.0055 - val_acc: 0.9981
Epoch 5/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0093 - acc: 0.9971 - val_loss: 0.0100 - val_acc: 0.9978
Epoch 6/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0064 - acc: 0.9979 - val_loss: 0.0018 - val_acc: 0.9996
Epoch 7/10
4154/4154 [==============================] - 28s 7ms/step - loss: 0.0055 - acc: 0.9984 - val_loss: 0.0050 - val_a